## E03 Predict water features given model
#### 0. Load required libraries, site dependant constants and utility functions.

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import seaborn as sns
import joblib
from sklearn import metrics

# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

#### 1. Load model

In [7]:
# uncomment to load the model and replace with your file, usually just correct the date
model_path = './LGBMClassifier_model_20000_unbalanced_WATER_only_MODIS_test_0_fixed_lr_01_min_wat_50.pkl'
model = joblib.load(model_path)

# predict the test labels
#plabels_test = model.predict(features_test)

#### 2. Define EOTask

In [ ]:
class PredictPatch(EOTask):
    """
    Task to make model predictions on a patch. Provide the model and the feature, 
    and the output names of labels and scores (optional)
    """
    def __init__(self, model, features_feature, predicted_labels_name, return_eopatch=True):
        self.model = model
        self.features_feature = features_feature
        self.predicted_labels_name = predicted_labels_name
        self.return_eopatch = return_eopatch
        
    def execute(self, eopatch):
        ftrs = eopatch[self.features_feature[0]][self.features_feature[1]]
        
        t, w, h, f = ftrs.shape
        ftrs = ftrs.reshape(t * w * h, f)
        
        plabels = self.model.predict(ftrs)
        plabels = plabels.reshape(t, w, h )
        plabels = plabels[..., np.newaxis]
        eopatch.add_feature(FeatureType.MASK, self.predicted_labels_name, plabels)
        
        #if self.predicted_scores_name:
        #    pscores = self.model.predict_proba(ftrs)
        #    _, d = pscores.shape
        #    pscores = pscores.reshape(w, h, d)
        #    eopatch.add_feature(FeatureType.DATA_TIMELESS, self.predicted_scores_name, pscores)
        
        if self.return_eopatch == True:
            return eopatch
        else:
            return plabels

In [ ]:
class CountMaskCoverage(EOTask):
    
    def __init__(self, input_mask_name, output_scalar_name ):
        self.input_mask_name = input_mask_name
        self.output_scalar_name = output_scalar_name
        
    def execute(self, eopatch):
        ultimate_masks = eopatch.mask[self.input_mask_name]
        mask_shape = ultimate_masks.shape
        #print(mask_shape)
        coverage_list = []
        
        for i in range(mask_shape[0]):
            array = ultimate_masks[i].squeeze()
            #print(array.shape)
            coverage = np.sum(array.astype(np.uint8)) #/ np.prod(array.shape)
            coverage_list.append(coverage)
        
        
        eopatch.add_feature(FeatureType.SCALAR_TIMELESS, self.output_scalar_name, np.asarray(coverage_list))
        return eopatch

In [ ]:
# TASK FOR PREDICTION
predict_patch = PredictPatch(model, (FeatureType.DATA, 'DATASET'), 'PREDICTED_WATER_FEATURES')

calculate_water_mask_coverage = CountMaskCoverage(input_mask_name='PREDICTED_WATER_FEATURES',
                                                            output_scalar_name='PREDICTED_WATER_MASK_COVERAGE')


#### 3. Check out EOTasks:

In [4]:
file = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[0], site.SITE_NAME, site.AVAILABLE_CRSs[0], site.DATE_RANGES_MODIS[0])
eo_patch = io_utils.load_exemplary_eopatch_from_file(file)

Loaded from ../../data/EOPatches/MODIS/UPE_PROMICE/UTM_22N/2013_04-2013_11/2975_maxcc_0.05_x-3_y-136


In [5]:
eo_patch = predict_patch.execute(eo_patch)
calculate_water_mask_coverage(eo_patch)

NameError: name 'predict_patch' is not defined

In [ ]:
plot_utils.plot_RGB_MODIS_image(eo_patch, datetime_idx=99)

In [ ]:
plot_utils.plot_RGB_MODIS_image(eo_patch, datetime_idx=100)

In [ ]:
data = eo_patch.mask['PREDICTED_WATER_FEATURES']

In [ ]:
data.shape

In [ ]:
plot_utils.plot_matrix(data[99])

In [ ]:
plot_utils.plot_matrix(data[100])

In [70]:
eo_patch.scalar_timeless['PREDICTED_WATER_MASK_COVERAGE']

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,  4779,    31,     0,   472,
           3,   458,   808,  1320,  1308,   697,   352,   308,   317,
         290,   290,   828,   305,   113,   116,    90,     0,     0,
           0,    22,     4,   253,   200,   284,   349,   108,   108,
          64,   173,   140,   136,   166,     0,    24,    56,     0,
          26,    13,     9,   663,    92,   260,   191,   174,   180,
           0,     0,     0,     0,     0,     0,     0,     0,   431,
         686,   868,     1,     1,     1,     1,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,    32,
           0,     0,

#### 2. Create and execute EOWorkflow

In [6]:
site.DATE_RANGES_MODIS[7:-2]

[('2000-04-26', '2000-11-05'),
 ('2001-04-26', '2001-11-05'),
 ('2002-04-26', '2002-11-05'),
 ('2003-04-26', '2003-11-05'),
 ('2004-04-26', '2004-11-05'),
 ('2005-04-26', '2005-11-05'),
 ('2006-04-26', '2006-11-05'),
 ('2007-04-26', '2007-11-05'),
 ('2008-04-26', '2008-11-05'),
 ('2009-04-26', '2009-11-05'),
 ('2010-04-26', '2010-11-05')]

In [78]:
%%time
# application dataset building eotasks for all aval eopatches in the a eoworkflow
for date_range_modis in site.DATE_RANGES_MODIS[7:-2]:
    print('Reading EOPatches from:', date_range_modis)
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[0], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range_modis)
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    predict_patch,
    calculate_water_mask_coverage,
    save_eotask
    )
                      
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    execution_args = []
    for eopatch_name in list_of_available_patches:
        execution_args.append({
        load_eotask: {'eopatch_folder': eopatch_name},
        save_eotask: {'eopatch_folder': eopatch_name}
        })
    
    #print(execution_args)                            
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=1, multiprocess=False)

    executor.make_report()

Reading EOPatches from: ('2000-04-26', '2000-11-05')



Reading EOPatches from: ('2001-04-26', '2001-11-05')



Reading EOPatches from: ('2002-04-26', '2002-11-05')



Reading EOPatches from: ('2003-04-26', '2003-11-05')



Reading EOPatches from: ('2004-04-26', '2004-11-05')


...


Reading EOPatches from: ('2007-04-26', '2007-11-05')



Reading EOPatches from: ('2008-04-26', '2008-11-05')



Reading EOPatches from: ('2009-04-26', '2009-11-05')



Reading EOPatches from: ('2010-04-26', '2010-11-05')



Wall time: 5h 39min 43s


**History:** Wall time: 8h 33min 55s